### Initialize connection and retrieve list of tables

In [ ]:
# courtesy of https://stackoverflow.com/questions/62340498/open-database-files-db-using-python
# loading in modules
import sqlite3
import pandas as pd

# Accessing local db
dbfile = 'databases/Global_Coral_Bleaching_Database_SQLite_11_24_21.db'

# Getting connection and cursor
con = sqlite3.connect(dbfile)
cur = con.cursor()

### List the names of all the tables (also see table_info.txt for column names in every table)

In [ ]:
table_list = [a for a in cur.execute("SELECT name FROM sqlite_master WHERE type = 'table'")]
print(type(table_list[0][0]))
print(*table_list, sep='\n')

<class 'str'>
('Authors_LUT',)
('Bleaching_Level_LUT',)
('Bleaching_Prevalence_Score_LUT',)
('Bleaching_tbl',)
('City_Town_Name_LUT',)
('Country_Name_LUT',)
('Cover_tbl',)
('Data_Source_LUT',)
('Ecoregion_Name_LUT',)
('Environmental_tbl',)
('Exposure_LUT',)
('Ocean_Name_LUT',)
('R_Scripts_tbl',)
('Realm_Name_LUT',)
('Sample_Event_tbl',)
('Severity_Code_LUT',)
('Site_Info_tbl',)
('State_Island_Province_Name_LUT',)
('Substrate_Type_LUT',)
('Query_2_Sample_Event_Counts',)
('Query_3_Time_Series',)
('Query_4_Samples_by_Data_Source',)
('Query_5_Sites_by_Data_Source',)
('Query_6_Sites_with_Multiple_Sample_Events',)
('Subquery_1_Sites_and_Sample_Events',)
('Subquery_6_Calculated_Reef_Check_Segments',)


### For printing specific table content

In [ ]:
TABLE_NAME='Bleaching_tbl'
table_bleaching = pd.read_sql_query(f"SELECT Bleaching_ID, Sample_ID, Percent_Bleached from {TABLE_NAME} WHERE Percent_Bleached IS NOT Null", con)
print(table_bleaching.shape)
table_bleaching.head(10)

(8961, 3)


,Bleaching_ID,Sample_ID,Percent_Bleached
0,31828,10275339,78.00
1,31985,10275496,42.00
2,32043,10275554,100.00
3,40828,10290570,90.15
4,40829,10290571,81.25
5,40830,10290572,96.46
6,40831,10290573,68.38
7,40832,10290574,100.00
8,40833,10290575,93.94
9,40834,10290576,69.84


### Count number of samples collected per ocean

In [ ]:
site_info="Site_Info_tbl"
ocean_id = "Ocean_Name_LUT"

oceans_count_df = pd.read_sql_query("SELECT Ocean_name, COUNT(*) FROM {} GROUP BY Ocean_name".format(site_info), con)
oceans_count_df=oceans_count_df.rename(columns={"Ocean_Name":"Ocean_ID"})

oceans_id_df = pd.read_sql_query("SELECT * from {}".format(ocean_id), con)
ocean_join = pd.merge(oceans_count_df, oceans_id_df, how='inner', on='Ocean_ID')

print(ocean_join[['Ocean_Name', 'COUNT(*)']])



     Ocean_Name  COUNT(*)
0       Pacific      5625
1      Atlantic      7222
2        Indian      1199
3       Red Sea       231
4  Arabian Gulf       128


### Retrieve number of samples per ecoregion
* select top 3 ecoregions to construct the dataset for the workshop
* Results - (Ecoregion ID, Ecoregion name):
    * (6, Bahamas and Florida Keys)
    * (9, Belize and west Caribbean)
    * (45, Hispaniola, Puerto Rico and Lesser Antilles)

In [ ]:
site_info = "Site_Info_tbl"
sample_info="Sample_Event_tbl"
eco_info = 'Ecoregion_Name_LUT'

# Retrieve base dataframes
site_df = pd.read_sql_query("SELECT * from {}".format(site_info), con)
site_df=site_df.rename(columns={"Ecoregion_Name":"Ecoregion_ID"})

sample_df = pd.read_sql_query("SELECT * from {}".format(sample_info), con)

eco_lut = pd.read_sql_query("SELECT * from {}".format(eco_info), con)

# Link site info with ecoregion name
eco_site_join = pd.merge(site_df, eco_lut, how="inner", on="Ecoregion_ID")

# Link ecoregion name with samples
eco_site_sample_join = pd.merge(eco_site_join, sample_df, how="inner", on="Site_ID")
count_region=eco_site_sample_join.groupby(['Ecoregion_Name']).count().sort_values(by="Sample_ID", ascending=False)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  
    print(count_region[['Sample_ID']])


                                                    Sample_ID
Ecoregion_Name                                               
Bahamas and Florida Keys                                 3965
Belize and west Caribbean                                3466
Hispaniola, Puerto Rico and Lesser Antilles              2117
Sunda Shelf, south-east Asia                             1275
Sulu Sea                                                 1176
Central and northern Great Barrier Reef                  1098
Cuba and Cayman Islands                                   956
Jamaica                                                   759
South-east Philippines                                    709
Fiji                                                      689
North and central Red Sea                                 687
Society Islands, French Polynesia                         683
Netherlands Antilles and south Caribbean                  518
South Vietnam                                             487
Gulf of 

### Filter bleachinng table based on the top 3 ecoregions
Dataframe columns for a basic version of the model (we could create a dataframe with more columns for a more complex model):
* Ecoregion_ID
* Sample_ID
* Date (bring together Date_day, Date_month, Date_year)
* Bleaching prevalence score

In [ ]:
site_tbl = "Site_Info_tbl"
sample_tbl="Sample_Event_tbl"
bleaching_tbl = "Bleaching_tbl"
Environmental_tbl = "Environmental_tbl"

site_df = pd.read_sql_query("SELECT * from {}".format(site_tbl), con)
sample_df = pd.read_sql_query("SELECT * from {}".format(sample_tbl), con)
bleaching_df = pd.read_sql_query("SELECT * from {}".format(bleaching_tbl), con)

# 1 - join 'Site_Info_tbl' with 'Sample_Event_tbl' on Site_ID
site_sample_join = pd.merge(site_df, sample_df, how="inner", on="Site_ID")

# 2 - filter the resulted dataframe by the 3 top ecoregions (Ecoregion_Name in {6, 9, 45})
top_3 = {6}#, 9, 45}
top_ecoregions_df = site_sample_join[site_sample_join["Ecoregion_Name"].isin(top_3)]

# 3 - join dataframe from 2 with 'Bleaching_tbl' on Sample_ID
site_sample_bleach_join = pd.merge(top_ecoregions_df, bleaching_df, how="inner", on="Sample_ID")

# 4 - filter dataframe from 3 to only contain the columns mentioned above
basic_columns = ['Sample_ID', 'Site_ID','Percent_Bleached', 'Date_Day', 'Date_Month', 'Date_Year', 'Ecoregion_Name', 'Latitude_Degrees', 'Longitude_Degrees']
basic_df = site_sample_bleach_join[basic_columns]
basic_df = basic_df.dropna(how='any',axis=0) 

#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  
#    print(basic_df)
print(basic_df.shape)
basic_df.head()

(3460, 9)


,Sample_ID,Site_ID,Percent_Bleached,Date_Day,Date_Month,Date_Year,Ecoregion_Name,Latitude_Degrees,Longitude_Degrees
9,10323308,1213,10.000000,15.0,8.0,1991.0,6.0,23.796,-76.137
10,10323309,1214,8.000000,15.0,3.0,1994.0,6.0,24.650,-76.805
11,10323310,1214,33.000000,15.0,4.0,1993.0,6.0,24.650,-76.805
12,10323311,1214,8.000000,15.0,8.0,1992.0,6.0,24.650,-76.805
17,10323316,1217,0.833333,15.0,6.0,1998.0,6.0,24.170,-74.480


### Get latest data for each site

In [ ]:
import tqdm
site_ids = list(set(basic_df['Site_ID']))
print(len(site_ids))

#yearly_data_region_1 = pd.DataFrame([],columns=["Year","Percent_Bleached_Avg","Temperature_Kelvin_Avg", "Num_Samples"])
latest_bleaching_data = []
#df2 = {'First Name': 'Vikram', 'Last Name': 'Aruchamy', 'Country': 'India'}

#df = df.append(df2, ignore_index = True)

for site_id in tqdm.tqdm(site_ids):
    # get data from this site:
    all_bleaching_data = basic_df.loc[basic_df['Site_ID'] == site_id]
    all_bleaching_data = all_bleaching_data.sort_values(['Date_Year', 'Date_Month', 'Date_Day'], ascending=[False, False, False])
    most_recent_bleaching_data = all_bleaching_data.head(1)
    latest_bleaching_data.append(most_recent_bleaching_data)
    
    
latest_bleaching_data = pd.concat(latest_bleaching_data)

latest_bleaching_data.to_csv("latest_bleaching_data.csv")

In [ ]:
x = list(avg_yearly_data['Year'])
y_bleached = list(avg_yearly_data['Percent_Bleached_Avg'])
y_temp = list(avg_yearly_data['Temperature_Kelvin_Avg'])
print(x)
print(y_bleached)
print(y_temp)

In [ ]:
import altair as alt
import numpy as np
# Generate some random data
rng = np.random.RandomState(1)
#x = rng.rand(40) ** 2
#y = 10 - 1.0 / (x + 0.1) + rng.randn(40)
y=y_temp
source = pd.DataFrame({"x": x, "y": y})

# Define the degree of the polynomial fits
degree_list = [1, 2, 3, 4]

base = alt.Chart(source).mark_circle(color="black").encode(
        alt.X("x", scale=alt.Scale(domain=[1986, 2020])), alt.Y("y", scale=alt.Scale(domain=[300, 310]))
)

polynomial_fit = [
    base.transform_regression(
        "x", "y", method="poly", order=order, as_=["x", str(order)]
    )
    .mark_line()
    .transform_fold([str(order)], as_=["degree", "y"])
    .encode(alt.Color("degree:N"))
    for order in degree_list
]

alt.layer(base, *polynomial_fit)

In [ ]:
import altair as alt
import numpy as np
# Generate some random data
rng = np.random.RandomState(1)
#x = rng.rand(40) ** 2
#y = 10 - 1.0 / (x + 0.1) + rng.randn(40)
y=y_bleached
source = pd.DataFrame({"x": x, "y": y})

# Define the degree of the polynomial fits
degree_list = [1, 2, 3, 4]

base = alt.Chart(source).mark_circle(color="black").encode(
        alt.X("x", scale=alt.Scale(domain=[1986, 2020])), alt.Y("y", scale=alt.Scale(domain=[0, 50]))
)

polynomial_fit = [
    base.transform_regression(
        "x", "y", method="poly", order=order, as_=["x", str(order)]
    )
    .mark_line()
    .transform_fold([str(order)], as_=["degree", "y"])
    .encode(alt.Color("degree:N"))
    for order in degree_list
]

alt.layer(base, *polynomial_fit)

### TODO:
* Aggreagate data for each site -> create timeseries?
* Create categories of percent_bleached (start with 0 and 1)


In [ ]:
top_10 = basic_df['Site_ID'].value_counts()[:10]
print(top_10)

In [ ]:
site_1 = basic_df.loc[basic_df['Site_ID']==1340]
site_1
#region_1 = basic_df.loc[basic_df['Longitude_Degrees']<=-76.805]

In [ ]:
# Close connection
con.close()